# Testing OpenSmile Classes

Draft 1 of pipeline for creating a df, loading the generated openSMILE features, and running feature selection.

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import json
from tqdm import tqdm
sys.path.append("/home/ubuntu/MultiModalDeepFake")
from packages.LJDataLoader import LJDataLoader
from packages.SmileFeatureLoader import smileFeatureLoader
from packages.SmileFeatureSelector import smileFeatureSelectorBruteForce, smileFeatureSelectFromModel, smileUnivariateFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.feature_selection as fs

## Part 1: Creating the dataframe with the train-dev-test split

In [2]:
#load LJ 16khz data
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
loader = LJDataLoader(data_path=file_path)

In [3]:
#split data into train, dev, test
loader.splitData()

13055
7833 10444


In [4]:
loader.metadata.columns

Index(['index', 'Unnamed: 0', 'id', 'Main_ID', 'Secondary_ID', 'Real',
       'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge',
       'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow', 'ElevenLabs',
       'UberDuck', 'transcript_1', 'transcript_2', 'type'],
      dtype='object')

In [5]:
#get the column names
cols = list(loader.metadata.columns[5:15])
cols

['Real',
 'Full_Band_MelGan',
 'HifiGan',
 'MelGan',
 'MelGanLarge',
 'Multi_Band_MelGan',
 'Parallel_WaveGan',
 'Waveglow',
 'ElevenLabs',
 'UberDuck']

In [6]:
#generate the dataframe
#single_id entry=True breaks
df = loader.generateFinalDataFrame(real_col=cols[0], fake_cols=cols[1:])

In [7]:
df.shape

(130550, 6)

In [8]:
df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ018-0361,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ019-0139,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ008-0187,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ018-0223,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ016-0417,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [9]:
df['architecture'].value_counts()

Real                 13055
Full_Band_MelGan     13055
HifiGan              13055
MelGan               13055
MelGanLarge          13055
Multi_Band_MelGan    13055
Parallel_WaveGan     13055
Waveglow             13055
ElevenLabs           13055
UberDuck             13055
Name: architecture, dtype: int64

In [10]:
# sample 10000 rows from df for testing with balanced real and fake
#sampled_df = df.groupby('B', group_keys=False).apply(lambda x: x.sample(n=min_group_size))

#temporary solution for testing

test_df = df.groupby('label', group_keys=False).apply(lambda x: x.sample(n=500))

In [11]:
#check balance
test_df.type.value_counts()

train    605
test     205
dev      190
Name: type, dtype: int64

In [12]:
test_df.architecture.value_counts()

Real                 500
HifiGan               72
MelGanLarge           64
ElevenLabs            60
Multi_Band_MelGan     59
UberDuck              58
Full_Band_MelGan      55
MelGan                45
Parallel_WaveGan      44
Waveglow              43
Name: architecture, dtype: int64

In [13]:
test_df.label.value_counts()

0    500
1    500
Name: label, dtype: int64

## Side: Creating/Updating File Mappings JSON:

This JSON has the path to the collection of wavfiles as the key and the corresponding smile features as values. 

For now, each time a new set of wav file are generated for a new provider/architecture - they should be stored in a separate unique folderpath. The `SmileFeatureGenerator` class will convert these to the csv file that can then be used to create a new JSON entry. 

In [ ]:
file_mapping = dict()

#biden real
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/44KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-44KHz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/16KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-16KHz.csv'

#biden eleven labs
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/44Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-44Khz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/16Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-16Khz.csv'

#biden in the wild
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-Original.csv'
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-16000KHz.csv'

#ljspeech real
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-16000KHz.csv'

#ljspeech eleven labs
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-16000KHz.csv'

#ljspeech uberduck
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-16000KHz.csv'

#ljspeech melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-16000KHz.csv'

#ljspeech melgan large
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-16000KHz.csv'

#ljspeech multi band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-16000KHz.csv'

#ljspeech full band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-16000KHz.csv'

#ljspeech hifiGAN
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-16000KHz.csv'

#ljspeech parallel wavegan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-16000KHz.csv'

#ljspeech waveglow
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-16000KHz.csv'

#file_mapping

In [ ]:
cellEnabled=0

if cellEnabled:

    file_mappings = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/' + 'file_mappings.json' 

    with open(file_mappings, "w") as json_file:
        json.dump(file_mapping, json_file, indent=4)

## Part 2: Loading the openSMILE features

The openSMILE features are generated and stored in multiple `csv` files here - `data/OpenSmile/04-2023-OSF-feature-csvs`. This folder also stores the JSON file

In [14]:
#pass the test df and the smileFeatureLoader object does the merging
smFeatLoader = smileFeatureLoader(test_df)

100%|██████████| 10/10 [02:01<00:00, 12.14s/it]


In [15]:
#get the df for feature selection
fs_df = smFeatLoader.return_merged_df()

In [16]:
fs_df.shape

(1000, 6380)

In [17]:
fs_df.head()

,type,id,architecture,path,label,multiclass_label,duration(seconds),audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,dev,LJ009-0260,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,1.504,3.210858,0.216783,0.993007,...,7.969172,0.537269,2.800857,2.633814,13.429184,0.614214,151.788300,93.544685,144.491440,56.958157
1,test,LJ015-0023,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,8.563,2.311157,0.070671,0.391048,...,11.307250,0.555090,2.970577,2.979340,-19.979172,0.461164,119.856010,65.274330,124.644340,66.294090
2,train,LJ023-0066,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,5.220,2.309739,0.366990,0.937864,...,9.972246,0.690888,2.720463,2.722816,-19.999962,0.377124,119.469540,63.340397,112.896164,56.064102
3,train,LJ004-0105,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,7.414,2.110040,0.527248,0.514986,...,7.553504,0.534313,2.727876,2.737227,-19.964270,0.458944,119.058624,56.358753,115.973946,51.865910
4,train,LJ050-0060,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,9.538,1.902781,0.151163,0.139535,...,7.638598,0.497670,2.552455,2.558427,-19.991665,0.545857,113.729520,58.177570,110.945210,57.596600


## Part 3: Running feature selection

In [18]:
#get the columns that are not the 6373 features as metadata
metadata = list(fs_df.columns[:7])
metadata

['type',
 'id',
 'architecture',
 'path',
 'label',
 'multiclass_label',
 'duration(seconds)']

### BruteForce feature selection (Buggy and doesn't run):

Has an issue due to how architecture is currently set up. Need a mapping between real and fake architectures for the code to run properly. Gautham to discuss further with Romit.

In [19]:
#initialize the object
smFS_bf = smileFeatureSelectorBruteForce(df=fs_df, metadata=metadata, standardize=True, model=LogisticRegression())

Initializing data...
Standardizing features...
smileFeatureSelectorBruteForce object initialized.



__architecture column needs fixing (real should be mapped to the corresponding fake that is assigned to it__

In [20]:
smFS_bf.data.architecture.unique()

array(['Real', 'ElevenLabs', 'Parallel_WaveGan', 'UberDuck', 'HifiGan',
       'MelGan', 'Multi_Band_MelGan', 'MelGanLarge', 'Full_Band_MelGan',
       'Waveglow'], dtype=object)

In [21]:
smFS_bf.data.architecture.value_counts()

Real                 500
HifiGan               72
MelGanLarge           64
ElevenLabs            60
Multi_Band_MelGan     59
UberDuck              58
Full_Band_MelGan      55
MelGan                45
Parallel_WaveGan      44
Waveglow              43
Name: architecture, dtype: int64

### Select From Model

In [22]:
smFS_model = smileFeatureSelectFromModel(df=fs_df,
                                        metadata=['type', 'id', 'architecture', 'path', 'label', 'multiclass_label', 'duration(seconds)'],
                                        standardize = True,
                                        model=RandomForestClassifier())

Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



In [23]:
exp_df_2 = smFS_model.select_features(max_features=25, arch='all_archs', return_df=True)


Top 25 features selected from all_archs architecture.

1. pcm_RMSenergy_sma_quartile2
2. pcm_fftMag_spectralFlux_sma_stddev
3. pcm_fftMag_spectralEntropy_sma_lpc2
4. mfcc_sma[4]_lpc0
5. mfcc_sma[6]_lpgain
6. mfcc_sma[6]_lpc0
7. mfcc_sma[6]_lpc1
8. mfcc_sma[7]_lpgain
9. mfcc_sma[8]_lpc0
10. mfcc_sma[10]_lpgain
11. mfcc_sma[12]_lpc0
12. mfcc_sma[13]_lpgain
13. mfcc_sma_de[6]_lpgain
14. mfcc_sma_de[6]_lpc1
15. mfcc_sma_de[8]_lpgain
16. mfcc_sma_de[8]_lpc2
17. mfcc_sma_de[9]_lpgain
18. mfcc_sma_de[9]_lpc0
19. jitterLocal_sma_percentile1.0
20. jitterDDP_sma_flatness
21. jitterDDP_sma_quartile1
22. jitterDDP_sma_quartile2
23. jitterDDP_sma_percentile1.0
24. audspec_lengthL1norm_sma_rqmean
25. pcm_fftMag_spectralFlux_sma_peakMeanAbs


In [24]:
exp_df_2.head()

,type,id,architecture,path,label,multiclass_label,duration(seconds),pcm_RMSenergy_sma_quartile2,pcm_fftMag_spectralFlux_sma_stddev,pcm_fftMag_spectralEntropy_sma_lpc2,...,mfcc_sma_de[8]_lpc2,mfcc_sma_de[9]_lpgain,mfcc_sma_de[9]_lpc0,jitterLocal_sma_percentile1.0,jitterDDP_sma_flatness,jitterDDP_sma_quartile1,jitterDDP_sma_quartile2,jitterDDP_sma_percentile1.0,audspec_lengthL1norm_sma_rqmean,pcm_fftMag_spectralFlux_sma_peakMeanAbs
0,dev,LJ009-0260,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,1.504,0.060996,0.418733,0.025716,...,0.138888,0.965260,-1.860047,0.003598,0.429953,0.004462,0.013563,0.002438,0.997488,0.963522
1,test,LJ015-0023,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,8.563,0.047888,0.265508,0.208156,...,-0.021208,0.870025,-1.828410,0.001473,0.401082,0.003500,0.007872,0.001107,0.802554,0.697738
2,train,LJ023-0066,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,5.220,0.037776,0.177167,0.188617,...,-0.010521,0.741540,-1.847833,0.001426,0.438787,0.003260,0.007696,0.000759,0.807089,0.451510
3,train,LJ004-0105,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,7.414,0.035107,0.172776,0.323238,...,-0.034936,0.990298,-1.900016,0.001218,0.495432,0.004910,0.010289,0.000962,0.638135,0.460996
4,train,LJ050-0060,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,9.538,0.044952,0.235707,0.155720,...,0.087988,0.751940,-1.794040,0.001904,0.451818,0.003431,0.007436,0.001499,0.710726,0.621085


### Univariate Feature Selection:

In [25]:
smFS_univariate = smileUnivariateFeatureSelector(df=fs_df,
                                                metadata=['type', 'id', 'architecture', 'path', 'label', 'multiclass_label', 'duration(seconds)'],
                                                standardize = True)

Initializing data...
Standardizing features...
smileUnivariateFeatureSelector object initialized.



In [26]:
exp_df_3 = smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, arch='all_archs', return_df=True)


Top 25 features selected from all_archs architecture.

1. mfcc_sma[6]_lpc0
2. mfcc_sma[6]_lpc1
3. mfcc_sma[7]_lpgain
4. mfcc_sma[9]_lpgain
5. mfcc_sma[10]_lpgain
6. mfcc_sma[10]_lpc0
7. mfcc_sma[12]_lpgain
8. pcm_fftMag_spectralEntropy_sma_de_upleveltime50
9. pcm_fftMag_spectralEntropy_sma_de_lpc1
10. mfcc_sma_de[5]_lpgain
11. mfcc_sma_de[5]_lpc0
12. mfcc_sma_de[6]_lpc0
13. mfcc_sma_de[6]_lpc1
14. mfcc_sma_de[6]_lpc2
15. mfcc_sma_de[7]_lpc0
16. mfcc_sma_de[8]_lpc1
17. mfcc_sma_de[9]_lpgain
18. mfcc_sma_de[9]_lpc1
19. mfcc_sma_de[9]_lpc2
20. mfcc_sma_de[13]_lpgain
21. mfcc_sma_de[14]_lpc0
22. jitterDDP_sma_quartile1
23. jitterDDP_sma_percentile1.0
24. audspec_lengthL1norm_sma_rqmean
25. mfcc_sma[14]_peakMeanRel


In [27]:
exp_df_3.head()

,type,id,architecture,path,label,multiclass_label,duration(seconds),mfcc_sma[6]_lpc0,mfcc_sma[6]_lpc1,mfcc_sma[7]_lpgain,...,mfcc_sma_de[8]_lpc1,mfcc_sma_de[9]_lpgain,mfcc_sma_de[9]_lpc1,mfcc_sma_de[9]_lpc2,mfcc_sma_de[13]_lpgain,mfcc_sma_de[14]_lpc0,jitterDDP_sma_quartile1,jitterDDP_sma_percentile1.0,audspec_lengthL1norm_sma_rqmean,mfcc_sma[14]_peakMeanRel
0,dev,LJ009-0260,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,1.504,-1.458140,0.294377,13.706154,...,1.037667,0.965260,1.550542,-0.419666,0.435053,-1.743573,0.004462,0.002438,0.997488,-0.691467
1,test,LJ015-0023,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,8.563,-1.650147,0.659978,12.533196,...,1.315105,0.870025,1.297118,0.097991,0.585227,-1.863716,0.003500,0.001107,0.802554,-0.265414
2,train,LJ023-0066,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,5.220,-1.598264,0.519333,9.975176,...,1.431306,0.741540,1.315783,-0.003893,0.429883,-1.777453,0.003260,0.000759,0.807089,-0.463732
3,train,LJ004-0105,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,7.414,-1.764933,0.831072,12.636436,...,1.344727,0.990298,1.517885,-0.178515,0.585094,-1.790064,0.004910,0.000962,0.638135,-0.002557
4,train,LJ050-0060,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,9.538,-1.616009,0.600396,11.458459,...,1.285241,0.751940,1.297805,0.031830,0.475726,-1.771895,0.003431,0.001499,0.710726,-0.110715
